In [3]:
from z3 import *

An RTT-level aggregate model of the network for modelling Copa.

## SMT Representation

Units are chosen so that the propagation delay (RTTmin) = 1

### Network parameters
* $C$ - Link rate in packets per RTT
* $L$ - Buffer length in packets. Curently infinite
* $T$ - Number of rounds we work for
* $W_{init}$ - Initial congestion window

### The variables
* $q_t$ - Queue length at the beginning of round $t$
* $o_t$ - Cumulative number of transmission opportunities till time $t$ (not including $t$)
* $d_t$ - Standing queuing delay (in number of packets) calculated at the beginning of the current RTT. It is based on what happened between one and two RTTs ago. It falls in a range depending on how the non-deterministic server chooses to schedule transmissions within the RTT
* $w_t$ - Congestion window at the beginning of round $t$

### The constraints on the model
* $C * t <= o_t <= C * t + k$ - Fundamental constraint on the non-deterministic server
* $d_t \in \left[C * \frac{q_{t-2}}{C},  C * (\frac{q_{t-2}}{C} + 1)\right]$ - it is a range because the non-deterministic server may choose to schedule packets in various ways

In [4]:
s = Solver()

# Declare the network parameters
C = 10
T = 10

# Declare the variables
q = [Int('q_%d' % t) for t in range(T)]
o = [Int('o_%d' % t) for t in range(T)]
d = [Int('d_%d' % t) for t in range(T)]
w = [Int('w_%d' % t) for t in range(T)]

for t in range(1, T):
    # How the queuing delay is updated. All packets arrive in the beginning of the RTT
    q_len = q[t - 1] + w[t] - o[t]
    #s.add(Or(q_len >= 0, q[t] == 0))
    #s.add(Or(q_len < 0, q[t] == q_len))
    s.add(q[t] == If(q_len >= 0, q_len, 0))
    
    # Standing queuing delay
    if t == 1:
        s.add(d[t] = 0)
    else:
        s.add(d[t] <= q[t-2] + C)
        s.add(d[t] <= )

# Non-aggregate analysis

## Parameters
* $T$ - number of timesteps to simulate
* $R$ - Propagation delay (in timesteps)
* $B$ - BDP (in packets)
* $K$ - Slack in the non-deterministic server
* $L$ - Buffer size (in packets)

## Variables
### Buffer
__Note:__ The buffer enqueues from i=L-1 and deques from i=0
* $v_{t,i}$ - Whether the $i^{th}$ position of the buffer has a packet at timestep $t$
* $e_{t,i}$ - Enque time of the packet in the $i^{th}$ position of the buffer at timestep $t$
* $l_{t,i}$ - If true, a packet immediately after this one was lost

### Non-deterministic server
* $o_t$ - True if there was a transmit opportunity at time $t$ (so packets will move in the buffer at $t+1$)
* $ops_t$ - Number of transmission opportunities so far. $ops_t = \sum_{t'=0}^{t-1} o_{t'}$

__Note:__ Constraint is that $\frac{B}{R} t \le ops_t \le \frac{B}{R} t + K$

### Congestion window
* $p_t$ - Whether a packet was sent at time $t$. Will be enqueued at $t+1$
* $w_t$ - Congestion window at time $t$
* $inflight_t$ - number of packets in flight at time $t$ (right after $t-1$ but not including what happened at $t$)

### Congestion Control (Copa)
* $d_t$ - The standing queuing delay at time $t$. The value is valid every $R/2$ steps. For any integer $n$, $d_{n \frac{R}{2}} = \min_{t \in \left[(n - 1)\frac{R}{2}, n \frac{R}{2}\right]} (queue\:delay\:for\:packet\:acked\:at\:t)$

In [144]:
s = Solver()

# Constants
R = 20
T = R * 2
B = 10
K = 5
L = 10

# Create the variables
v, e, l = [], [], []
for i in range(T):
    v += [[Bool('v_%d,%d' % (t, i)) for t in range(L)]]
    e += [[Int('e_%d,%d' % (t, i)) for t in range(L)]]
    l += [[Bool('l_%d,%d' % (t, i)) for t in range(L)]]

o = [Bool('o_%d' % t) for t in range(T)]
ops = [Int('ops_%d' % t) for t in range(T)]

p = [Bool('p_%d' % t) for t in range(T)]
w = [Int('w_%d' % t) for t in range(T)]
inflight = [Int('inflight_%d' % t) for t in range(T)]

d = [Int('d_%d' % t) for t in range(T)]

### Start creating the constraints

# Constraints in the buffer
for t in range(1, T):
    # Are we moving?
    move = o[t-1] == True

    # Packets move in the buffer if there is a tx opp at time t-1. Else they don't
    for i in range(L-1):        
        # Are we adding a packet in position i?
        if i != 0:
            add_pkt = And(p[t-1] == True, 
                          Or(And(move,      v[t-1][i] == True, v[t-1][i+1] == False),
                             And(Not(move), v[t-1][i] == False, v[t-1][i-1] == True)))
        else:
            add_pkt = And(p[t-1] == True,
                          Or(And(move,      v[t-1][1] == False),
                             And(Not(move), v[t-1][0] == False)))
        
        # They move
        s.add(Implies(And(move, Not(add_pkt)),
                      And(v[t][i] == v[t-1][i+1],
                          e[t][i] == e[t-1][i+1],
                          l[t][i] == l[t-1][i+1])))
        
        # They don't move
        s.add(Implies(And(Not(move), Not(add_pkt)), 
                      And(v[t][i] == v[t-1][i],
                          e[t][i] == e[t-1][i],
                          l[t][i] == l[t-1][i])))
        
        # Add packet if needed
        s.add(Implies(add_pkt, And(v[t][i] == True, 
                                   e[t][i] == t,
                                   l[t][i] == False)))

    ## Handle the last position in the buffer
    # If buffer is full, it moves, and a packet comes in: add it in
    s.add(Implies(And(move, p[t-1] == True, v[t-1][-1] == True),
                  And(v[t][-1] == True,
                      e[t][-1] == t,
                      l[t][-1] == False)))
    # If buffer is full, it doesn't move, and a packet comes in: mark loss
    s.add(Implies(And(Not(move), p[t-1] == True, v[t-1][-1] == True),
                  And(v[t][-1] == True,
                      e[t][-1] == e[t-1][-1],
                      l[t][-1] == True)))

    # If a packet doesn't come in, and it moves: mark last position as empty
    s.add(Implies(And(move, p[t-1] == False),
                  And(v[t][-1] == False,
                      e[t][-1] == 0,
                      l[t][-1] == False)))
    # Maintain previous state in all other cases
    s.add(Implies(And(Not(move), Or(p[t-1] == False, v[t-1][-1] == False)),
                  And(v[t][-1] == v[t-1][-1],
                      e[t][-1] == e[t-1][-1],
                      l[t][-1] == l[t-1][-1])))
    s.add(Implies(And(move, v[t-1][-1] == False),
                  And(v[t][-1] == v[t-1][-1],
                      e[t][-1] == e[t-1][-1],
                      l[t][-1] == l[t-1][-1])))

for t in range(1, T):
    ### Constraints on the non-deterministic server
    # Update ops
    s.add(Implies(o[t-1] == True,  ops[t] == ops[t-1] + 1))
    s.add(Implies(o[t-1] == False, ops[t] == ops[t-1]))
    
    # Bounds on ops
    lbound = int(float(t) * B / R)
    ubound = int(float(t) * B / R) + K
    
    # Constraint on o based on ops
    #s.add(Implies(ops[t] == ubound,
    #              o[t] == False))
    #s.add(Implies(ops[t] == lbound,
    #              o[t] == True))
    
    # Constraints on ops
    s.add(ops[t] <= ubound)
    s.add(ops[t] >= lbound)
    
    ### Implement the congestion window
    ## Update inflight
    if t >= R:
        # Whether a packet was acked (i.e. sent from server 1 RTT ago)
        acked = And(v[t - R][0] == True, o[t - R] == True)
        # Decrement
        s.add(Implies(And(acked, p[t-1] == False),
                      inflight[t] == inflight[t-1] - 1))
        # Increment
        s.add(Implies(And(Not(acked), p[t-1] == True),
                      inflight[t] == inflight[t-1] + 1))
        # No change
        s.add(Implies(Or(And(acked, p[t-1] == True),
                         And(Not(acked), p[t-1] == False)),
                      inflight[t] == inflight[t-1]))
    else:
        acked = False

        # Increment
        s.add(Implies(p[t-1] == True,
                      inflight[t] == inflight[t-1] + 1))
        # No change
        s.add(Implies(p[t-1] == False,
                      inflight[t] == inflight[t-1]))
    
    ## Set p based on inflight and w
    s.add(p[t] == (inflight[t] < w[t]))
    
    ### Implement Copa. Runs every RTT/2
    assert(R % 2 == 0)
    if t > R:
        delay = t - e[t-R][0] - R
        if t % (R / 2) == 0:
            # Reset
            s.add(d[t] == If(acked, delay, R + L + 1))
            # Standing queuing delay is valid now. Change cwnd
            increase = (d[t-1] * w[t-1] <= 2 * R)
            s.add(w[t] == If(increase, w[t-1] + 1, w[t-1] - 1))
        else:
            # Update standing queuing delay
            s.add(d[t] == If(acked, 
                             If(delay < d[t-1], delay, d[t-1]),
                             d[t-1]))
            # Keep cwnd constant
            s.add(w[t] == w[t-1])
    else:
        s.add(d[t] == R + L + 1)
        # Keep cwnd constant
        s.add(w[t] == w[t-1])
        
### Initial conditions
for i in range(0, L):
    s.add(v[0][i] == False)
    s.add(e[0][i] == False)
    s.add(l[0][i] == False)

s.add(ops[0] == 0)
s.add(inflight[0] == 0)
s.add(w[0] == 15)
s.add(d[0] == R + L + 1)

### The question we want to ask
# See if server can prevent cwnd from increasing
s.add(w[-1] > 15)

In [145]:
s.check()

sat

In [147]:
def display(s):
    m = s.model()
    for t in range(T):
        line = "{:<4}".format(t)
        line += ('    ', 'Send')[int(bool(m[p[t]]))]
        line += " " * 5
        
        for i in range(L):
            i = L - i - 1
            if m[v[t][i]]:
                line += "{:3}".format(m[e[t][i]].as_long())
            else:
                line += "  *" #" " * 3
        
        line += "  :  "
        
        if m[o[t]]:
            line += "Served"
        else:
            line += "      "
        
        line += " " * 5 + "cwnd={:<3} inflight={:<3} ops={:<3} d={:<3}".format(
            m[w[t]].as_long(),
            m[inflight[t]].as_long(),
            m[ops[t]].as_long(),
            m[d[t]].as_long())

        print(line)
display(s)

0   Send       *  *  *  *  *  *  *  *  *  *  :  Served     cwnd=15  inflight=0   ops=0   d=31 
1   Send       *  *  *  *  *  *  *  *  *  1  :             cwnd=15  inflight=1   ops=1   d=31 
2   Send       *  *  *  *  *  *  *  *  2  1  :             cwnd=15  inflight=2   ops=1   d=31 
3   Send       *  *  *  *  *  *  *  3  2  1  :  Served     cwnd=15  inflight=3   ops=1   d=31 
4   Send       *  *  *  *  *  *  *  4  3  2  :  Served     cwnd=15  inflight=4   ops=2   d=31 
5   Send       *  *  *  *  *  *  *  5  4  3  :             cwnd=15  inflight=5   ops=3   d=31 
6   Send       *  *  *  *  *  *  6  5  4  3  :             cwnd=15  inflight=6   ops=3   d=31 
7   Send       *  *  *  *  *  7  6  5  4  3  :  Served     cwnd=15  inflight=7   ops=3   d=31 
8   Send       *  *  *  *  *  8  7  6  5  4  :  Served     cwnd=15  inflight=8   ops=4   d=31 
9   Send       *  *  *  *  *  9  8  7  6  5  :             cwnd=15  inflight=9   ops=5   d=31 
10  Send       *  *  *  * 10  9  8  7  6  5  :  Se